Загружаем данные

In [1]:
import requests

with open('Lab1.zip', 'wb') as fh:
  resp = requests.get('https://github.com/DimaKurd/cv_lab1/raw/master/lab1/Lab1.zip')
  fh.write(resp.content)

In [2]:
! unzip Lab1.zip -d .

Archive:  Lab1.zip
   creating: ./images/
  inflating: ./images/Abyssinian_122.jpg  
  inflating: ./images/Abyssinian_19.jpg  
  inflating: ./images/Abyssinian_210.jpg  
  inflating: ./images/Abyssinian_24.jpg  
  inflating: ./images/Abyssinian_31.jpg  
  inflating: ./images/Abyssinian_44.jpg  
  inflating: ./images/Abyssinian_66.jpg  
  inflating: ./images/Abyssinian_9.jpg  
  inflating: ./images/american_bulldog_140.jpg  
  inflating: ./images/american_bulldog_199.jpg  
  inflating: ./images/american_bulldog_95.jpg  
  inflating: ./images/basset_hound_118.jpg  
  inflating: ./images/basset_hound_14.jpg  
  inflating: ./images/basset_hound_181.jpg  
  inflating: ./images/beagle_1.jpg   
  inflating: ./images/beagle_180.jpg  
  inflating: ./images/Bengal_103.jpg  
  inflating: ./images/Bengal_147.jpg  
  inflating: ./images/Bengal_50.jpg  
  inflating: ./images/Birman_31.jpg  
  inflating: ./images/Birman_7.jpg   
  inflating: ./images/Bombay_16.jpg  
  inflating: ./images/Bombay_48.jp

# Functions

In [20]:
import os
import time

import numpy as np
import pandas as pd
import cv2

from scipy import signal
from tqdm import tqdm
from typing import List

import warnings
warnings.filterwarnings("ignore")

Метод для обработки изображения

In [21]:
class CannyEdgeDetector (object):
    def get_img_edges(self, img, threshold):
        grad_magnitute, grad_degree = self.get_mag_degree(img)
        supressed = self.non_max_supression(grad_magnitute, grad_degree)
        thresholded = self.threshold(supressed, threshold)
        output = self.edge_tracking(thresholded)
        
        return output

    @staticmethod
    def non_max_supression(mag, gdegree):
        height, width = mag.shape
        suppresed = np.zeros_like(mag)

        for x in range(1, width-1):
            for y in range(1, height-1):

                # Проверяем соседние пиксели в зависимости от направления градиента
                # 8 соседей - значит шаг по углу pi/8 = 180 / 8
                if (180* 1/8 <= gdegree[y][x] <= 180* 3/8) or (180* 9/8 < gdegree[y][x] <= 180* 11/8):
                    first_neibour = mag[y-1][x+1]
                    second_neibour = mag[y+1][x-1]

                elif (180* 3/8 < gdegree[y][x] <= 180* 5/8) or (180* 11/8 < gdegree[y][x] <= 180* 13/8):
                    first_neibour = mag[y-1][x]
                    second_neibour = mag[y+1][x]

                elif (180* 5/8 < gdegree[y][x] <= 180* 7/8) or (180* 13/8 < gdegree[y][x] <= 180* 15/8):
                    first_neibour = mag[y-1][x-1]
                    second_neibour = mag[y+1][x+1]

                else:
                    first_neibour = mag[y-1][x-1]
                    second_neibour = mag[y+1][x+1]

                if mag[y][x] > first_neibour and mag[y][x] > second_neibour:
                    suppresed[y][x] = mag[y][x]

        return suppresed

    @staticmethod
    def threshold(img, threshold):
        img[np.where((img >= threshold))] = 255
        img[np.where(img < threshold)] = 0

        return img

    @staticmethod
    def edge_tracking(img, weak=125):

        height, width = img.shape
        # используем blob-analysis

        for i in range(0, height):
            for j in range(0, width):
                if img[i][j] == weak:
                    if ((img[i + 1][j] == 255) or 
                        (img[i - 1][j] == 255) or 
                        (img[i][j + 1] == 255) or 
                        (img[i][j - 1] == 255) or 
                        (img[i + 1][j + 1] == 255) or 
                        (img[i - 1][j - 1] == 255)):
                        img[i][j] = 255
                    else:
                        img[i][j] = 0

        return img

    @staticmethod
    def get_mag_degree(img):
        Lx = np.array([[1, 0, -1],
                    [2, 0, -2],
                    [1, 0, -1]])
        Ly = np.array([[1, 2, 1],
                    [0, 0, 0],
                    [-1, -2, -1]])
        G = np.array([[2, 4, 5, 4, 2],
                    [4, 9, 12, 9, 4],
                    [5, 12, 15, 12, 5],
                    [4, 9, 12, 9, 4],
                    [2, 4, 5, 4, 2]])
        G = G / 159
        img_blured = signal.convolve2d(img, G, mode='same')
        magx = signal.convolve2d(img_blured, Lx, mode='same')
        magy = signal.convolve2d(img_blured, Ly, mode='same')

        mag = np.sqrt(magx**2 + magy**2)
        degree = np.rad2deg(np.arctan2(magy, magx))

        return mag, degree



In [22]:
canny = CannyEdgeDetector()
def get_foreground_mask(image_path: str) -> List[tuple]:
    """
    Метод для вычисления маски переднего плана на фото
    :param image_path - путь до фото
    :return массив в формате [(x_1, y_1), (x_2, y_2), (x_3, y_3)], в котором перечислены все точки, относящиеся к маске
    """

    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ###
    t1 = time.time()
    ###
    threshold = 25
    img_edges = canny.get_img_edges(img, threshold)

    img = np.where(img_edges > 1,1,0).astype('uint8')
    pred_points = np.argwhere(img)
    pred_points[:, [1, 0]] = pred_points[:, [0, 1]]
    cv2.fillPoly(img, [pred_points], 1)
    ###
    t2 = time.time()
    ###
    pred_points = np.argwhere(img)

    return pred_points, (t2-t1) / (img.shape[0] * img.shape[1]) #* 1e6

Расчёт метрики

In [23]:
def evaluate_iou(image_dir: str, anno_dir: str) -> float:
    """
    Метод для расчёта mean IoU на датасете
    :param image_dir - каталог с фото для анализа
    :param anno_dir - каталог с аннотациями для расчета метрики
    :return значение mean IoU с сохранением оценки по каждому фото в csv-файл
    """

    iou_data = dict()
    for _, _, files in os.walk(image_dir):
        assert files is not None, 'no files read'
        for image_name in tqdm(files):
            # print(f'Processing file {image_name}')
            sample_name = image_name[:image_name.find('.jpg')]

            # acquiring ground truth mask data
            mask_true = cv2.imread(os.path.join(anno_dir, f'{sample_name}.png'))
            assert mask_true is not None, 'mask is None'
            true_points = cv2.cvtColor(mask_true, cv2.COLOR_BGR2GRAY)
            true_points[true_points != 2] = 1
            true_points[true_points == 2] = 0
            true_points = np.argwhere(true_points)
            true_points_set = set([tuple(x) for x in true_points])

            # acquiring predicted mask
            pred_points, it_time = get_foreground_mask(image_path=os.path.join(image_dir, image_name))
            assert pred_points is not None, 'pred_points is None'
            pred_points_set = set([tuple(x) for x in pred_points])

            # calculating IoU
            iou = len(true_points_set.intersection(pred_points_set)) / len(true_points_set.union(pred_points_set))

            image_names = iou_data.get('image_names', [])
            image_names.append(sample_name)
            iou_data['image_names'] = image_names

            iou_values = iou_data.get('iou_values', [])
            iou_values.append(iou)
            iou_data['iou_values'] = iou_values
            iou_data['iter_time'] = it_time

        pd.DataFrame(data=iou_data).to_csv('detailed_results.csv')
        return np.mean(iou_data['iou_values']), np.mean(iou_data['iter_time'])

Вызов расчёта метрики

In [24]:
print(f"Metric_value {evaluate_iou(image_dir='./images', anno_dir='./annotations')}")

100%|██████████| 100/100 [00:41<00:00,  2.41it/s]

Metric_value (0.751127928086716, 1.2488378681474637e-06)
